In [1]:
#To confirm what we really need...
import pickle

from bs4 import BeautifulSoup
import regex as re
from nltk.stem.porter import PorterStemmer

In [2]:
import pandas as pd

In [14]:
import streamlit as st

In [3]:
with open('model_base.pkl', 'rb') as f: #reading bytes
    foo = pickle.load(f)

In [4]:
trial_model = foo

In [5]:
def document_words(raw_document):

    document_text = BeautifulSoup(raw_document).get_text()
    
    letters_only = re.sub("[^a-zA-Z0-9]", " ", document_text)
    
    words = letters_only.split()
    words = [w.lower() for w in words]
 
    p_stemmer = PorterStemmer()
    stemmed_words = [p_stemmer.stem(i, to_lowercase=False) for i in words]

    return(" ".join(stemmed_words))

IE insert here the stuff for h streamlit to gather the following:

In [23]:
raw_comment = 'bro mang you a stanky bagel boi zionist'
raw_comment2 = "lul I'm sure after he fired off all those loyal employees he was drinking their blood and tears on Passover #Zionist"
raw_comment3 = 'AND HE STILL DOESNT HAVE ENOUGH MONY TO FIX HIS UGLY GIANT ZIONIST NOSE???'

In [27]:
user1 = "bro mang you a stanky bagel boi zionist;lul I'm sure after he fired off all those loyal employees he was drinking their blood and tears on Passover #Zionist;AND HE STILL DOESNT HAVE ENOUGH MONY TO FIX HIS UGLY GIANT ZIONIST NOSE???"

In [31]:
# posts = len(user1.split(';'))
# posts

3

In [41]:
user1_logs=user1.split(';')

In [33]:
# user1_logs=[
#     {'comment':raw_comment,'occur':1}
#     ,{'comment':raw_comment2,'occur':2}
#     ,{'comment':raw_comment3,'occur':3}
# ]

End of streamlit userinput<br>
<Br>
 finally we do the following:

In [44]:
sample = pd.DataFrame({'cleaned_comments':user1_logs})
sample['cleaned_comments'] = sample['cleaned_comments'].apply(document_words)

In [60]:
preds = trial_model.predict(sample['cleaned_comments'])
preds_prob = trial_model.predict_proba(sample['cleaned_comments'])
sample_pred_comparer = pd.DataFrame({
    #'cleaned_text':X_test
    #,'actual':y_test
    'pred':preds
    ,'p_0':preds_prob[:,0]
    ,'p_1':preds_prob[:,1]
    ,'p_2':preds_prob[:,2]
})
sample_pred_comparer['max_prob'] = sample_pred_comparer[['p_0','p_1','p_2']].max(axis=1)
sample_pred_comparer['1_prob'] = [sample_pred_comparer.loc[i, 'p_0'] if sample_pred_comparer.loc[i, 'pred'] == 0
                                                      else (1-sample_pred_comparer.loc[i, 'p_0'])
                                                      for i in range(sample_pred_comparer.shape[0])]

#The idea behind the 1_prob is as follows: A statement that's actually a 2 is just as toxic as a 1, so in a way the probability
#that it is AT LEAST a 1 would be p1 +p2.So toof row a predicted 1: There's still a probability that it's actually a 2, so
#we should be able to add that probability back to p_1 to reflect a more solid liklihood that the post is at least a 1 -
#as once again, anything htat's a 2 is also a 1. Hence, add them...

#sample_pred_comparer['occur'] = [user1_logs[i]['occur'] for i in range(len(user1_logs))]
sample_pred_comparer

,pred,p_0,p_1,p_2,max_prob,1_prob
0,1,0.138888,0.685333,0.175779,0.685333,0.861112
1,2,0.133519,0.216752,0.649729,0.649729,0.866481
2,1,0.150192,0.687045,0.162763,0.687045,0.849808


In [ ]:
sample_pred_comparer['1_prob'] = [
    sample_pred_comparer.loc[i, 'max_prob'] if sample_pred_comparer.loc[i, 'pred'] != 1
    else (1 - sample_pred_comparer.loc[i, 'p_0'])
    for i in range(sample_pred_comparer.shape[0])

In [54]:
sample_pred_comparer.shape[0]

3

In [46]:
def prob_scanner_calc(p_dominant, category):
    if category!=0:
        q = 1-p_dominant #HMMMM this is technically not true, iewe need it 0 for it to not be actionable... ie so
        #it should b in the 2 case: 1 - p2 -p1... or at least a fraction of p1... we'll keep it at ease though
        return round(q, 3)
    else: #It actually is a 0, ie the liklihood of seeing htis many comments and it's benign
        return round(p_dominant, 3)

In [49]:
def prob_scanner_account(df):
    overall = 1
    for i in range(df.shape[0]):
        overall *= prob_scanner_calc(df.iloc[i]['max_prob'],df.iloc[i]['pred'])
    return overall #Ie the probability that all these posts are benign

In [97]:
#We'lldo 2s first then 1s

def prob_scanner_account(df, thresh_2=.58, thresh_1=.94, one_response=3):
    overall = 1
    count_2s = df['pred'].value_counts()[2]
    thresh_2 -= .05 * count_2s
    if df[df['max_prob']==df['max_prob'].max()].iloc[0]['p_2'] > thresh_2 and df[df['max_prob']==df['max_prob'].max()].iloc[0]['pred'] == 2:
        return "calling the principal AND your guardians"
    count_1s = df['pred'].value_counts()[1] + df['pred'].value_counts()[2]
    thresh_1 -= .03 * count_1s
    if df[df['pred']!=0]['1_prob'].mean() > thresh_1 and count_1s >= one_response:
        return "calling the principal"

In [98]:
prob_scanner_account(sample_pred_comparer)

'calling the principal'

In [67]:
df = sample_pred_comparer.copy()
df.head()

,pred,p_0,p_1,p_2,max_prob,1_prob
0,1,0.138888,0.685333,0.175779,0.685333,0.861112
1,2,0.133519,0.216752,0.649729,0.649729,0.866481
2,1,0.150192,0.687045,0.162763,0.687045,0.849808


In [74]:
df['pred'].value_counts()[2]

1

In [76]:
df[df['max_prob']==df['max_prob'].max()]

,pred,p_0,p_1,p_2,max_prob,1_prob
2,1,0.150192,0.687045,0.162763,0.687045,0.849808


In [92]:
df[df['max_prob']==df['max_prob'].max()].iloc[0]['p_2']

0.16276300492892715

In [94]:
df[df['pred']!=0]['1_prob'].mean()

0.8591339751964998

In [ ]:
df[df['max_prob']==df['max_prob'].max()]

In [52]:
# python -m streamlit run app_malevonce.py

In [51]:
# streamlit run app_malevonce.py

In [ ]:
#User should be threshed...

Hmm, now we gotta figur out probability stuff:

Theoretically,our tolerance for hate speach, albit for a review, would be with one two. For ones, let's say 2.

We'll work on the 2 case first:

pn2 off the bae, yet 1-pn2 if it's not.Then, we se a second post
that one has pn22 in of its own right, then 1-pn2 that it's not.


Hmm, I see a few things:
m1 - hte pure 2 approch, ie if probabilities of it being that noneof ht posts contain a 2,ieprobability among all the posts
that at least 1 is a 2,then review them: 
    However, as time goes on.... 
Def,and usre eough some are, mins are a third... 

Approach1 - Average the guys...perhaps have a guy to increase nominally as n increases.... ie a 90 for one; 85 for 2...
    Re. 1s - .95, .91, .87...
    
Using the example above: Hmm, I'd say all those would call,not for level 2, but at least level one...

Re. fushions: Hmm, perhaps I could just say p2 includes p1...


NOOOO I don't like it. Ie then I'd always be assuming to take action per the basis of my model being right...

Well, tthen again, it is a sliding scale - we assum accuracy, but a combination of of givng the benefit of the doubtand a continual

Gah, I might just need to call it here for now and bank on detection. Furthermore, recall: The median for both 1s and 2s are .55 and .58....

Re 2s:Given that a definite 2 would warrant action we could say: If pred2 p's is greater than .58 then we call in.
If two, then it'd be .55